In [3]:
using Distributions
using Rocket
using ReactiveMP
using BenchmarkTools

import Base: show

In [4]:
function kalman_filter_graph()
    model = Model(DefaultMessageGate())
    
    x_prior = add!(model, datavar(:x_prior, Normal{Float64}))
    add_1   = add!(model, constvar(:add_1, 1.0))    
    noise   = add!(model, constvar(:noise, Normal(0.0, sqrt(200.0))))
    
    x = add!(model, randomvar(:x))
    y = add!(model, datavar(:y, Float64))
    
    x_prev_add      = add!(model, make_node(+, x_prior, add_1, x))
    add_x_and_noise = add!(model, make_node(+, x, noise, y)) 
    
    activate!(model, x_prev_add)
    activate!(model, add_x_and_noise)
    
    return x_prior, x, y
end

function kalman(data)
    N = length(data)
    
    x_prior, x, y = kalman_filter_graph2()

    link(x_prior, getmarginal(x))
    
    update!(x_prior, Normal(0.0, sqrt(10000.0)))
    
    marginals = Vector{Normal{Float64}}(undef, N)
    
    subscribe!(getmarginal(x) |> enumerate(), (t) -> marginals[t[1]] = getdata(t[2]))
    
    for d in data
        update!(y, d)
    end
    
    return marginals
end

kalman (generic function with 1 method)

In [5]:
N = 600
data = collect(1:N) + sqrt(200.0) * randn(N);

In [8]:
@time kalman(data); # Initial compilation

  0.001106 seconds (16.36 k allocations: 817.484 KiB)


In [9]:
@time kalman(data); # Subsequent runs

  0.001080 seconds (16.36 k allocations: 817.484 KiB)


In [10]:
@btime kalman($data) # Performance benchmark

  650.944 μs (16362 allocations: 817.48 KiB)


600-element Array{Normal{Float64},1}:
 Normal{Float64}(μ=12.023014779420963, σ=3.757946762270414)
 Normal{Float64}(μ=9.709367522218457, σ=2.6582082959958475)
 Normal{Float64}(μ=10.27631371701221, σ=2.1706736357085332)
 Normal{Float64}(μ=14.785729290517441, σ=1.8799692411628124)
 Normal{Float64}(μ=19.458892227794703, σ=1.6815550395263243)
 Normal{Float64}(μ=20.56173040317447, σ=1.5350788837432578)
 Normal{Float64}(μ=21.943611752585824, σ=1.4212308075340512)
 Normal{Float64}(μ=20.429942923491744, σ=1.3294564709682384)
 Normal{Float64}(μ=21.327860232804902, σ=1.2534358890063568)
 Normal{Float64}(μ=21.34286478899955, σ=1.189123034279198)
 Normal{Float64}(μ=21.316775310552526, σ=1.1337916144768012)
 Normal{Float64}(μ=23.091963572072473, σ=1.0855286405502962)
 Normal{Float64}(μ=24.896947342841443, σ=1.0429469095396076)
 ⋮
 Normal{Float64}(μ=589.3830284502654, σ=0.15495276495869575)
 Normal{Float64}(μ=590.4074879826759, σ=0.1548213936816792)
 Normal{Float64}(μ=591.4035220589609, σ=0.154690355

In [ ]:
}